## Programming Assignment 2 - Part 1
### Cpts 355 - Fall 2015
### An Interpreter for a Postscript-like Language

### Assigned Sept. 11, 2015

### Due Wednesday, Sept. 23, 2015
Develop your code in a file named `sps.ipynb`, starting from this notebook file. When you are finished, upload `sps.ipynb` on the course Turnin Page. 

The entire interpreter project (Parts 1 and Part 2 together) will count for 10% of your course grade. This first part is worth 20% of that 10%: the intention is to make sure that you are on the right track and have a chance for mid-course correction before completing Part 2. However, note that the work and amount of code involved in Part 1 is a large fraction of the total project, so you need to get going on this part right away.

### This assignment is to be your own work. Refer to the course academic integrity statement in the syllabus.

## The problem
In this assignment you will write an interpreter in Python for a small PostScript-like language, concentrating on key computational features of the abstract machine, omitting all PS features related to graphics, and using a somewhat-simplified syntax.

The simplified language, SPS, has the following features of PS
* integer constants, e.g. `123`: in python3 there is no practical limit on the size of integers
* boolean constants, `true` and `false` (Note that the boolean constants in python are `True` and `False`)
* name constants, e.g. `/fact`: start with a `/` and letter followed by an arbitrary sequence of letters and numbers
* names to be looked up in the dictionary stack, e.g. `fact`: as for name constants, without the `/`
* code constants: code between matched curly braces `{` ... `}`
* built-in operators on numbers: `add`, `sub`, `mul`, `div`, `eq`, `lt`, `gt`
* built-in operators on boolean values: `and`, `or`, `not`; these take boolean operands only. Anything else is an error.
* built-in sequencing operators: `if`, `ifelse`; make sure that you understand the order of the operands on the stack. Play with ghostscript if necessary to help understand what is happening.
* stack operators: `dup`, `exch`, `pop`
* dictionary creation operator: `dictz`; takes no operands
* dictionary stack manipulation operators: `begin`, `end`. `begin` requires one dictionary operand on the operand stack; `end` has no operands.
* name definition operator: `def`. This requires two operands, a name and a value
* defining (using `def`) and calling functions
* stack printing operator (prints contents of stack without changing it): `stack`
* top-of-stack printing operator (pops the top element of the stack and prints it): `=`


## Requirements for Part 1 (Due Sept. 23)
In Part 1 you will build some essential pieces of the interpreter but not yet the full interpreter. The pieces you build will be driven by Python test code rather than actual Postscript programs. The pieces you are going to build first are:
* The operand stack
* The dictionary stack
* The operators that don't involve code arrays: all of the operators except `if`, `ifelse`.
In Part 2 we will add the implementations for `if`, `ifelse`, calling functions, as well as interpreting input strings in the Postscript language.
* Looking up names

### The operand stack
The operand stack should be implemented as a Python list. The list will contain **Python** integers, booleans, and strings. Python integers and booleans on the stack represent Postscript integers and booleans. Python strings on the stack represent names of Postscript variables (see the handling of names and the `def` operator below.

When using a list as a stack one of the decisions you have to make is where the *hot* end of the stack is located. (The *hot* end is where pushing and popping happens). Will the hot end be at position `0`, the head of the list, or at position `-1`, the end of the list? It's your choice.

### The dictionary stack
The dictionary stack is also implemented as a Python list. It will contain **Python** dictionaries which will be the implementation for **Postscript** dictionaries. The dictionary stack needs to support adding and removing dictionaries at the hot end, as well as defining and looking up names. 

### Operators
Operators will be implemented as zero-argument Python functions that manipulate the operand and dictionary stacks. For example, the `add` operator could be implemented as the Python function
```
def add():
    op1 = # pop the top value off the operand stack
    op2 = # pop the top value off the operand stack
    # push (op1 + op2) onto the operand stack
```
You may run into conflicts for some of the names of these functions . For example, the function for the `not` operator can't be named `not` because it is reserved for another use in Python. So you could do something like:
```
def psnot():
    // pop the top value off the operand stack and push its negation onto the operand stack
```    
The `begin` and `end` operators are a little different in that they manipulate the dictionary stack in addition to or instead of the operand stack. Remember that the `dictz` operator affects *only* the operand stack.

The `def` operator takes two operands from the operand stack: a string (recall that strings in the operand stack represent names of postscript variables) and a value. It changes the dictionary at the hot end of the dictionary stack so that the string is mapped to the value by that dictionary. Notice that `def` does ***not*** change the number of dictionaries on the dictionary stack!

### Name lookup

Name lookup is implemented by a Python function:
```
def lookup(name):
    # search the dictionaries on the dictionary stack starting at the hot end to find one that contains name
    # return the value associated with name
```
Note that name lookup is ***not*** a Postscript operator, but it ***is*** implemented by a Python function.

## Your Code Start Here

In [143]:
# The operand stack: define the operand stack and its operations in this notebook cell
opstack = []

# now define functions to push and pop values on the opstack according to your decision about which
# end should be the hot end.

# Pops an operator. Default pop(0) from the top
def opPop(): 
    if len(opstack) > 0:
        return opstack.pop()
    pass

# Pushes value to opstack
def opPush(value):
    opstack.append(value)
    pass

# Remember that there is a Postscript operator called "pop" so we choose different names for these functions.


In [144]:
# The dictionary stack: define the dictionary stack and its operations in this cell
dictstack = [{}]
# now define functions to push and pop dictionaries on the dictstack, to define name, and to lookup a name

# Pops an entry in dictstack. Default pop(0) from the top
def dictPop():
    if len(dictstack) > 0:
        return dictstack.pop() # default pop(0) from the top
    pass

# Pushes value to dictstack
def dictPush(value):
    dictstack.append(value)
    pass

# Pops name and value off opstack, sets var name and value association then pushes it onto dictstack
# **Renamed psDef according to given test cases
def psDef():
    value = opPop()    
    name = opPop()
    opPush(name) # Put it back
    opPush(value)
    # Case: empty stack
    if not dictstack: begin()
    newdict = dictPop()
    newdict[name] = value
    dictPush(newdict)
    pass

# Traverses dictstack, returns v=0 if there is no definition for name
def lookup(name):
    v = 0
    for each in dictstack:
        v = each.get(name, 0)
        if v != 0:
            return v
    return v
    pass

    # return the value associated with name
    # what is your design decision about what to do when there is no definition for name

In [145]:
# Arithmetic operators: define all the arithmetic operators in this cell -- add, sub, mul, div, eq, lt, gt

# Adds first two items on opstack
def add():
    first = opPop()
    second = opPop()
    opPush(first + second)

# Subtracts first two items on opstack
def sub():
    first = opPop()
    second = opPop()
    opPush(first - second)

# Multiplies first two items on opstack
def mul():
    first = opPop()
    second = opPop()
    opPush(first * second)

# Divides the first item on opstack by the second item
def div():
    first = opPop()
    second = opPop()                    
    opPush(first / second)

# True if first item equals second item
def eq():
    first = opPop()
    second = opPop() 
    opPush(first == second)

# True if first item < second item
def lt():
    first = opPop()
    second = opPop() 
    opPush(first > second)

# True if first item > second item
def gt():
    first = opPop()
    second = opPop() 
    opPush(first < second)  



In [146]:
# Boolean operators: define all the boolean operators in this cell -- and, or, not

# Logical bitwise and (both must be 1 to be true)
def andd():
    first = opPop()
    second = opPop()
    opPush(first and second)

# Logical bitwise or (one must be 1 to be true)
def orr():
    first = opPop()
    second = opPop()
    opPush(first or second)

# Reverses whatever the value on opstack was
def nott():
    opPush(not opPop())

In [147]:
# Define the stack manipulation operators in this cell: dup, exch, pop

# Pops a value off opstack and pops it back on twice
def dup():
    first = opPop()
    opPush(first)
    opPush(first)

# Pops values off opstack and pops them on in reverse order
def exch():  
    first = opPop()
    second = opPop()
    opPush(first)
    opPush(second)

# (see above) Pops an operator. Default pop(0) from the top
# def opPop(): 
#     if len(opstack) > 0:
#         return opstack.pop()
#     pass

In [148]:
# Define the dictionary manipulation operators in this cell: dictz, begin, end, def
# name the function for the def operator psDef because def is reserved in Python

# Creates and returns a new dictionary
def dictz():
    newdict = {}
    return newdict

# Appends a new dictionary to dictstack
def begin():
    tmpdict = dictz()
    dictstack.append(tmpdict)

# Removes a dictionary from dictstack
def end():
    return dictstack.pop()

# (see above) Pops name and value off opstack, sets var name and value association
# then pushes it onto dictstack
# def psDef():
#    value = opPop()    
#    name = opPop()
#    opPush(name) # Put it back
#    opPush(value)
#    # Case: empty stack
#    if not dictstack: begin()
#    newdict = dictPop()
#    newdict[name] = value
#    dictPush(newdict)
#    pass

## Comment
* `dictz` should push a dict on the `opstack`; `begin` will pop the dict from `opstack` and push to `dictstack`.

In [149]:
# Define the printing operators in this cell: =, stack
# Pick a good name for the code implementing =

def pop_and_print():
    first = opstack.pop
    print(first)
    
def stack():
    for each in opstack:
        print(each)

A typo
`first = opstack.pop()`


In [150]:
# Define any other operators that I may have forgotten in this cell

# ? Unclear

## Test your code
With all of that stuff defined, you will be able to test your interpreter using Python code like this:

In [151]:
def testAdd():
    opPush(34)
    opPush(16)
    add()
    if opPop() != 50: return False   
    return True

def testLookup():
    opPush("n1")
    opPush(3)
    psDef()
    if lookup("n1") != 3: return False
    return True

def testDictPush_Pop():
    dictPush({"Hello": 400})
    if dictPop() != {"Hello": 400}: return False
    return True

def testLt():
    opPush(40)
    opPush(50)
    lt()
    if opPop() != True: return False
    return True

def testGt():
    opPush(50)
    opPush(40)
    gt()
    if opPop() != True: return False
    return True

def testDup():
    opPush(12);
    dup()
    first = opPop()
    second = opPop()
    if (first != 12) or (second != 12): return False
    return True

def testExch():
    opPush(12);
    opPush(24);
    exch()
    first = opPop()
    second = opPop()
    if (first != 12) and (second != 24): return False
    return True

def testBegin_Dictz():
    dictstack.pop()
    begin()
    if not dictstack: return false
    return True

def testEnd():
    dictstack.pop()
    begin()
    end()
    if not dictstack: return True
    return False

# not called in testAll, output not shown
def testpopandprint():
    opPush(54)
    pop_and_print()
    
# not called in testAll, output not shown    
def teststack():
    opPush(4)
    opPush(2)
    opPush(0)
    stack()

# go on writing test code for ALL of your code here; think about edge cases, and 
# other points where you are likely to make a mistake.

# now an easy way to run all the test cases and make sure that they all return true
# is

testCases = [testAdd, testLookup]
def testAll():
    for test in testCases:
        if not test(): return False
    return True

# but wouldn't it be nice to run all the tests, instead of stopping on the first failure,
# and see which ones failed
# How about something like:

testCases = [('add', testAdd), ('lookup', testLookup), ('lt', testLt), ('gt', testGt), ('dup', testDup), ('exch', testExch), ('begin', testBegin_Dictz), ('end', testEnd)]
def testAll():
    failedTests = [testName for (testName, testProc) in testCases if not testProc()]
    if failedTests:
        return ('Some tests failed', failedTests)
    else: return ('All tests OK')

In [152]:
testAll()

'All tests OK'

## Summary
## FinalScore = 97
* Check the comments above and see the `dictz` and `begin`.
* The ideas on stacks are good.
* The code is elegant.